# 16. Transform Orders Data - Explode Array

In [0]:
%sql
SELECT * FROM gizmobox.silver.orders_json;

## 1. Access elemet JSON Object

In [0]:
%sql
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  json_value.items
 FROM gizmobox.silver.orders_json;

## 2. Deduplicate array elements

In [0]:
%sql
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  array_distinct(json_value.items) as items
 FROM gizmobox.silver.orders_json;

## 3. Explode Arrays

In [0]:
%sql
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) as items
 FROM gizmobox.silver.orders_json;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW tv_orders_explodes
AS
SELECT 
  json_value.order_id,
  json_value.order_status,
  json_value.payment_method,
  json_value.total_amount,
  json_value.transaction_timestamp,
  json_value.customer_id,
  explode(array_distinct(json_value.items)) as items
 FROM gizmobox.silver.orders_json;

In [0]:
%sql
SELECT 
    order_id,
    order_status,
    payment_method,
    total_amount,
    transaction_timestamp,
    customer_id,
    items.item_id,
    items.name,
    items.price,
    items.quantity,
    items.category,
    items.details.brand,
    items.details.color
FROM tv_orders_explodes

## Create Silver Table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gizmobox.silver.orders
SELECT 
    order_id,
    order_status,
    payment_method,
    total_amount,
    transaction_timestamp,
    customer_id,
    items.item_id,
    items.name,
    items.price,
    items.quantity,
    items.category,
    items.details.brand,
    items.details.color
FROM tv_orders_explodes

In [0]:
%sql
SELECT * FROM gizmobox.silver.orders